0. library

In [ ]:
pip install pycaret
pip install -U "numpy<1.20"
pip install bayesian-optimization

In [13]:
import pandas as pd

from pycaret.classification import *

import numpy as np
from sklearn.model_selection import StratifiedKFold, cross_val_score
from lightgbm import LGBMClassifier
from sklearn.preprocessing import MinMaxScaler, StandardScaler

from sklearn.metrics import log_loss
import time
from bayes_opt import BayesianOptimization

from datetime import datetime
import os

1. loading data

In [28]:
data1 = pd.read_csv("/content/drive/MyDrive/unidthon data/train1.csv")
data2 = pd.read_csv("/content/drive/MyDrive/unidthon data/train2.csv")
data3 = pd.read_csv("/content/drive/MyDrive/unidthon data/train3.csv")
data4 = pd.read_csv("/content/drive/MyDrive/unidthon data/train4.csv")
data5 = pd.read_csv("/content/drive/MyDrive/unidthon data/train5.csv")

train = pd.concat([data1, data2, data3, data4, data5])

data1 = pd.read_csv("/content/drive/MyDrive/unidthon data/val1.csv")
data2 = pd.read_csv("/content/drive/MyDrive/unidthon data/val2.csv")
data3 = pd.read_csv("/content/drive/MyDrive/unidthon data/val3.csv")
data4 = pd.read_csv("/content/drive/MyDrive/unidthon data/val4.csv")
data5 = pd.read_csv("/content/drive/MyDrive/unidthon data/val5.csv")

val = pd.concat([data1, data2, data3, data4, data5])

test = pd.read_csv("/content/drive/MyDrive/unidthon data/test.csv")

train = train.drop(['site'], axis=1)
val = val.drop(['site'], axis=1)
test = test.drop(['site', 'sid'], axis=1)
train['leaktype'].replace(['out','in','noise','other','normal'], [0,1,2,3,4], inplace=True)
val['leaktype'].replace(['out','in','noise','other','normal'], [0,1,2,3,4], inplace=True)
test['leaktype']=""

# train을 target과 feature로 나눠줍니다.
train_x=train.drop(['leaktype'], axis=1)
train_y=train[['sid', 'leaktype']]
val_x=val.drop(['leaktype'], axis=1)
val_y=val[['sid', 'leaktype']]
test_x=test.drop(['leaktype'], axis=1)
test_y=test['leaktype']

In [29]:
train_x.head()

,sid,C01,C02,C03,C04,C05,C06,C07,C08,C09,C10,C11,C12,C13,C14,C15,C16,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26
0,S-0359369085120315,19,30,27,26,24,14,10,9,6,15,9,9,13,13,20,16,24,20,24,38,60,198,185,46,17,4680
1,S-0359369083968368,2,343,46,22,12,8,6,4,4,2,2,4,2,2,2,2,2,2,2,2,2,4,2,2,2,300
2,S-0359369085133797,0,36,5,3,3,0,3,3,3,3,3,3,3,0,3,3,0,0,3,3,3,3,3,3,3,240
3,S-0359369084117593,2,24,4,4,2,2,2,4,2,4,4,4,4,4,2,4,2,4,2,4,4,4,4,2,4,370
4,S-0359369084010178,3,25,5,5,5,5,5,5,6,5,5,3,5,6,5,5,5,5,6,8,6,5,5,6,5,430


2. Feature Engineering

In [ ]:
x_pivot_train = pd.pivot_table(
    data = train_x,
    values = train_x.columns,
    index = 'sid',
    aggfunc = [
        'sum', 'mean', 'median',
        'min', 'max', 'std', 'var'
    ]
)

x_pivot_test = pd.pivot_table(data = val_x,
                              values = val_x.columns,
                              index = 'sid',
                              aggfunc = [
        'sum', 'mean', 'median',
        'min', 'max', 'std', 'var'
    ])

display(x_pivot_train.head)
display(x_pivot_test.head)

In [ ]:
x_columns = [agg + "_" + column for agg, column in x_pivot_train.columns]
x_pivot_train.columns = x_columns
x_pivot_test.columns = x_columns
display(x_pivot_train.head())

In [40]:
x_pivot_train.reset_index(inplace = True)
x_pivot_test.reset_index(inplace = True)

In [ ]:
train_y

In [41]:
train_data = pd.merge(x_pivot_train, train_y, on='sid')
train_data.label = train_data.leaktype.astype('category')

In [42]:
df = setup(data = train_data, target="leaktype", silent = True)

,Description,Value
0,session_id,4304
1,Target,leaktype
2,Target Type,Multiclass
3,Label Encoded,None
4,Original Data,"(62564, 184)"
5,Missing Values,True
6,Numeric Features,182
7,Categorical Features,1
8,Ordinal Features,False
9,High Cardinality Features,False


INFO:logs:create_model_container: 0
INFO:logs:master_model_container: 0
INFO:logs:display_container: 1
INFO:logs:Pipeline(memory=None,
         steps=[('dtypes',
                 DataTypes_Auto_infer(categorical_features=[],
                                      display_types=False, features_todrop=[],
                                      id_columns=[],
                                      ml_usecase='classification',
                                      numerical_features=[], target='leaktype',
                                      time_features=[])),
                ('imputer',
                 Simple_Imputer(categorical_strategy='not_available',
                                fill_value_categorical=None,
                                fill_value_numerical=None,
                                numeric_st...
                ('scaling', 'passthrough'), ('P_transform', 'passthrough'),
                ('binn', 'passthrough'), ('rem_outliers', 'passthrough'),
                ('cluste

In [ ]:
compare_models(exclude=['gbc'], sort = 'Accuracy', n_select = 1, fold = 4)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
rf,Random Forest Classifier,0.9968,1.0000,0.9958,0.9968,0.9968,0.9958,0.9958,16.7300
et,Extra Trees Classifier,0.9967,0.9999,0.9957,0.9967,0.9967,0.9956,0.9956,29.9475
lightgbm,Light Gradient Boosting Machine,0.9966,0.9998,0.9959,0.9966,0.9966,0.9956,0.9956,26.0725
ridge,Ridge Classifier,0.9961,0.0000,0.9953,0.9962,0.9961,0.9949,0.9949,5.4300
dt,Decision Tree Classifier,0.9958,0.9972,0.9954,0.9958,0.9958,0.9944,0.9944,4.8400
qda,Quadratic Discriminant Analysis,0.9905,0.9946,0.9924,0.9912,0.9906,0.9875,0.9876,139.6250
knn,K Neighbors Classifier,0.9877,0.9986,0.9869,0.9877,0.9877,0.9838,0.9838,45.5500
ada,Ada Boost Classifier,0.5692,0.8167,0.4663,0.5439,0.5439,0.4243,0.4310,35.7875
lr,Logistic Regression,0.4963,0.7199,0.3495,0.5454,0.4203,0.2933,0.3268,482.8050
lda,Linear Discriminant Analysis,0.4839,0.7009,0.3527,0.4973,0.4207,0.2741,0.2976,90.7625


INFO:logs:create_model_container: 13
INFO:logs:master_model_container: 13
INFO:logs:display_container: 2
INFO:logs:RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=4304, verbose=0,
                       warm_start=False)
INFO:logs:compare_models() succesfully completed......................................


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=4304, verbose=0,
                       warm_start=False)

In [ ]:
best_score = 0
columns = 'scaled' + x_pivot_train.columns
for scaler in [None, MinMaxScaler(), StandardScaler()]:
  if scaler == None:
    scaled_train_x = x_pivot_train.copy()
    scaled_test_x = x_pivot_test.copy()
  else:
    scaled_train_x = scaler.fit_transform(x_pivot_train)
    scaled_train_x = pd.DataFrame(scaled_train_x, columns = columns).astype('float64')
    scaled_test_x = scaler.transform(x_pivot_test)
    scaled_test_x = pd.DataFrame(scaled_test_x, columns = columns).astype('float64')
  
  skf = StratifiedKFold(n_splits=3, shuffle=True)
  model = LGBMClassifier(n_jobs = -1)
  scores = cross_val_score(model, x_pivot_train, train_y.label.values, scoring='accuracy', cv=skf, n_jobs=-1)

  if scaler == None: print("NON_SCALED")
  else:
    print(scaler.__class__.__name)
  print(f'TOTAL 최대성능: {max(scores)}\n평균성능: {np.mean(scores)}')
  print("-"*30)

  if np.mean(scores) >= best_score:
    best_score = np.mean(scores)
    best_train_x = scaled_train_x
    best_test_x = scaled_test_x
    best_scaler = scaler

if best_scaler == Nonde:
  print("BEST SCALER IS NON_SCALED")
else:
  print("BEST SCALER: ", best_scaler.__class__.__name__)

In [ ]:
display(best_train_x.head())

In [ ]:
lgbm_pbounds = {
    'learning_rate': (0.001, 0.5),
    'max_depth': (5,20),
    'n_estimators': (100, 300)
}

def lgbm_opt(learning_rate, max_depth, n_estimators):
  params = {
      'learning_rate': learning_rate,
      'max_depth': int(round(max_depth)),
      'n_estimators': int(round(max_depth))
  }
  lgbm = LGBMClassifier(**params)
  score = 100 - abs(cross_val_score(lgbm, best_train_x, train_y.leaktype.values, scoring="neg_log_loss", cv=4, n_jobs=-1).mean())
  return score

BO_lgbm = BayesianOptimization(f = lgbm_opt, pbounds = lgbm_pbounds, randoms_state = 1)

BO_lgbm.maximize(init_points = 20, n_iter = 20)

max_params = BO_lgbm.max['params']
max_params['max_depth'] = int(round(max_params['max_depth']))
max_params['n_estimators'] = int(round(max_params['n_estimators']))
print(max_params)

model_lgbm = LGBMClassifier(**max_params)
model_lgbm.fit(best_train_x, train_y.leaktype.values, eval_metric = 'logloss')
prediction = model_lgbm.predict_proba(best_test_x)

In [ ]:
os.mkdir(path + 'submission')
sample_submission = pd.read_csv(path + 'sample_submission.csv')
sample_submission.iloc[:,1:] = prediction
score = abs(cross_val_score(model_lgbm, best_train_x, train_y.leaktype.values, scoring = "neg_log_loss", cv=4, n_jobs = -1).mean())
score = str(round(score, 2)).replace(".", "")
print(score)